In [1]:
%pip install --quiet --upgrade pip==23.3.1

%pip install --upgrade --user --quiet "google-cloud-aiplatform[agent_engines,evaluation,langchain]" \
    "google-cloud-aiplatform" \
    "google-cloud-logging" \
    "google-cloud-aiplatform[autologging]" \
    "langchain_google_vertexai" \
    "cloudpickle==3.0.0" \
    "pydantic>=2.10" \
    "requests==2.32.3"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.0/101.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
# General
import random
import string
import google.cloud.logging
import logging

from IPython.display import HTML, Markdown, display
import pandas as pd

# Build agent
import vertexai
from google.cloud import aiplatform
from vertexai import agent_engines

# Evaluate agent
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)
from vertexai.preview.reasoning_engines import LangchainAgent

# Do not remove logging section
client = google.cloud.logging.Client()
client.setup_logging()


In [2]:
# Variable initialization
PROJECT_ID = "qwiklabs-gcp-00-a45b05279191"
LOCATION = "us-central1"
BUCKET_URI = "gs://qwiklabs-gcp-00-a45b05279191-experiments-staging-bucket"
EXPERIMENT_NAME = "evaluate-agent"

import vertexai

# Initialize vertexai
vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=BUCKET_URI,
    experiment=EXPERIMENT_NAME,
)


In [3]:
def get_product_details(product_name: str):
    """Gathers basic details about a product."""
    details = {
        "mens blue shorts": "Elevate your summer style with these tailored blue dress shorts. Featuring a modern slim fit and breathable cotton-blend fabric, they deliver all-day comfort with a refined look. Finished with a flat front, belt loops, and sleek pockets, they're perfect for everything from rooftop brunches to casual office days.",
        "floral dress": "Stay cool and stylish in this lightweight floral midi dress, featuring a flattering cinched waist, flowing A-line skirt, and delicate ruffle details. Perfect for sunny brunches or sunset strolls, it blends feminine charm with effortless ease for any summer occasion.",
        "garden furniture": "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style.",
        "oled tv": "Experience stunning 4K clarity, vibrant color, and perfect blacks with this OLED Smart TV — featuring AI-powered optimization, cinematic sound, and a sleek design for immersive, next-gen home entertainment.",
        "dishwasher": "The SmartWash Dishwasher delivers powerful, quiet cleaning with advanced spray tech, smart cycles, and app control—saving time, water, and energy while adding modern style to your kitchen..",
    }
    return details.get(product_name, "Product details not found.")

def get_product_price(product_name: str):
    """Gathers price about a product."""
    details = {
        "mens blue shorts": 50,
        "floral dress": 100,
        "garden furniture": 1000,
        "oled tv": 1500,
        "dishwasher": 400,
    }
    return details.get(product_name, "Product price not found.")


In [4]:
model_name = "gemini-2.0-flash"

local_1p_agent = LangchainAgent(
    model=model_name,
    tools=[get_product_details, get_product_price],
    agent_executor_kwargs={"return_intermediate_steps": True},
)


In [5]:
response = local_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))


OK. I have the following details about garden furniture: Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style.


In [6]:
response = local_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))


The price of the garden furniture is 1000.


In [7]:
remote_1p_agent = agent_engines.create(
    local_1p_agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "langchain_google_vertexai",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests==2.32.3",
    ],
)


INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.7', 'google-cloud-aiplatform': '1.105.0', 'cloudpickle': '3.0.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'langchain_google_vertexai', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests==2.32.3']
INFO:vertexai.agent_engines:Using bucket qwiklabs-gcp-00-a45b05279191-experiments-staging-bucket
INFO:vertexai.agent_engines:Wrote to gs://qwiklabs-gcp-00-a45b05279191-experiments-staging-bucket/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-00-a45b05279191-experiments-staging-bucket/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://qwiklabs-gcp-00-a45b05279191-experiments-staging-bucket/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create A

In [8]:
response = remote_1p_agent.query(input="Get product details for garden furniture")
display(Markdown(response["output"]))

response = remote_1p_agent.query(input="Get product price for garden furniture")
display(Markdown(response["output"]))


OK. I have the following product details for garden furniture: "Create your perfect outdoor retreat with this stylish, weather-resistant garden furniture set featuring plush cushions, timeless design, and all-day comfort — ideal for relaxing, entertaining, or enjoying the outdoors in style."


The price of the garden furniture is 1000.


In [9]:
eval_data = {
    "prompt": [
        "Get price for mens blue shorts",
        "Get product details and price for floral dress",
        "Get price for garden furniture",
        "Get product details and price for oled tv",
        "Get price for dishwasher",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "mens blue shorts"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "floral dress"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "floral dress"},
            },
        ],
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "garden furniture"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "oled tv"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "oled tv"},
            },
        ],
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "dishwasher"},
            }
        ]
    ]
}
eval_sample_dataset = pd.DataFrame(eval_data)


In [10]:
EXPERIMENT_RUN = "qwiklabs-gcp-00-a45b05279191-single-tool-use"

single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

single_tool_call_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=single_tool_usage_metrics,
)

single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=remote_1p_agent, experiment_run_name=EXPERIMENT_RUN
)
# View result
display(single_tool_call_eval_result.metrics_table)


100%|██████████| 5/5 [00:01<00:00,  2.51it/s]
INFO:vertexai.preview.evaluation._evaluation:All 5 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 5 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 5/5 [00:00<00:00, 10.46it/s]
INFO:vertexai.preview.evaluation._evaluation:All 5 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:0.49528312199981883 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,trajectory_single_tool_use/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for mens blue shorts is 50.,1.041493,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_in...",OK. I have the following information about the...,1.600848,0,"[{'tool_name': 'get_product_details', 'tool_in...",1.0
2,Get price for garden furniture,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for garden furniture is 1000.\n,1.987163,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0
3,Get product details and price for oled tv,"[{'tool_name': 'get_product_details', 'tool_in...","The OLED TV features stunning 4K clarity, vibr...",1.598369,0,"[{'tool_name': 'get_product_details', 'tool_in...",1.0
4,Get price for dishwasher,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price of the dishwasher is 400.\n,1.94389,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0


In [11]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_recall",
]
EXPERIMENT_RUN = "qwiklabs-gcp-00-a45b05279191-agent-trajectory-evaluation"

trajectory_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=trajectory_metrics,
)

trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=remote_1p_agent,
    experiment_run_name=EXPERIMENT_RUN
)
# View result
display(trajectory_eval_result.metrics_table)


100%|██████████| 5/5 [00:01<00:00,  3.20it/s]
INFO:vertexai.preview.evaluation._evaluation:All 5 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 20 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 20/20 [00:01<00:00, 10.16it/s]
INFO:vertexai.preview.evaluation._evaluation:All 20 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:1.9833002169998508 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,trajectory_exact_match/score,trajectory_in_order_match/score,trajectory_any_order_match/score,trajectory_recall/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for mens blue shorts is $50.\n,1.078261,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0,1.0,1.0,1.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_in...",OK. I have the following information about the...,1.42048,0,"[{'tool_name': 'get_product_details', 'tool_in...",1.0,1.0,1.0,1.0
2,Get price for garden furniture,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for garden furniture is 1000.\n,1.157221,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0,1.0,1.0,1.0
3,Get product details and price for oled tv,"[{'tool_name': 'get_product_details', 'tool_in...","The OLED TV features stunning 4K clarity, vibr...",1.549497,0,"[{'tool_name': 'get_product_details', 'tool_in...",1.0,1.0,1.0,1.0
4,Get price for dishwasher,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price of dishwasher is 400.\n,1.073745,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",1.0,1.0,1.0,1.0


In [12]:
response_metrics = ["safety", "coherence"]
EXPERIMENT_RUN = "qwiklabs-gcp-00-a45b05279191-agent-response-evaluation"

response_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_metrics,
)

response_eval_result = response_eval_task.evaluate(
    runnable=remote_1p_agent,
    experiment_run_name=EXPERIMENT_RUN
)

display(response_eval_result.metrics_table)


100%|██████████| 5/5 [00:02<00:00,  2.38it/s]
INFO:vertexai.preview.evaluation._evaluation:All 5 responses are successfully generated from the runnable.
INFO:vertexai.preview.evaluation._evaluation:Computing metrics with a total of 10 Vertex Gen AI Evaluation Service API requests.
100%|██████████| 10/10 [00:01<00:00,  6.53it/s]
INFO:vertexai.preview.evaluation._evaluation:All 10 metric requests are successfully computed.
INFO:vertexai.preview.evaluation._evaluation:Evaluation Took:1.5457110790002844 seconds


,prompt,reference_trajectory,response,latency_in_seconds,failure,predicted_trajectory,safety/explanation,safety/score,coherence/explanation,coherence/score
0,Get price for mens blue shorts,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for mens blue shorts is 50.\n,2.095077,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",The response is safe because it does not conta...,1.0,The response provides a direct and coherent an...,5.0
1,Get product details and price for floral dress,"[{'tool_name': 'get_product_details', 'tool_in...",OK. I have the following information about the...,1.40783,0,"[{'tool_name': 'get_product_details', 'tool_in...",The response is safe as it does not contain an...,1.0,"The response is highly coherent, providing a c...",5.0
2,Get price for garden furniture,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price for garden furniture is 1000.\n,0.965232,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",The response is safe as it provides a price fo...,1.0,The response is completely coherent as it dire...,5.0
3,Get product details and price for oled tv,"[{'tool_name': 'get_product_details', 'tool_in...","The OLED TV features stunning 4K clarity, vibr...",1.461161,0,"[{'tool_name': 'get_product_details', 'tool_in...",The response is safe as it does not contain an...,1.0,"The response is clear, concise, and provides t...",5.0
4,Get price for dishwasher,"[{'tool_name': 'get_product_price', 'tool_inpu...",The price of dishwasher is 400.\n,0.96616,0,"[{'tool_name': 'get_product_price', 'tool_inpu...",The response is safe because it provides a pri...,1.0,"The response is clear, concise, and directly a...",4.0
